In [236]:
#from elasticsearch_dsl import Search, Q, Range, connections
#from elasticsearch.client import Elasticsearch
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import re
from openai import OpenAI
import numpy as np

In [237]:
DATA_FOLDER = "../data/events/browse_ai/" 

In [426]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 50)
pd.set_option('display.max_colwidth', None)

In [239]:
df = pd.read_pickle(f"{DATA_FOLDER}df_browse_ai 2025-01-02_18-50-53.pkl")

In [ ]:
extracted_columns = []
for col in df.columns:
    if col.startswith("extr"):
        extracted_columns.append(col)
    
extracted_columns.append("isOnMultipleDays")
extracted_columns

In [241]:
id = 18075
df.loc[id, "isOnMultipleDays"]=False

In [242]:
id = 18072
df.loc[id, "isOnMultipleDays"]=False
df.loc[id, "extrStartTime"]="11"
df.loc[id, "extrEndTime"]="13"
df.loc[id, "extrMultipleDaysDetails"]=None

In [243]:
id = 4212
df.loc[id, "isOnMultipleDays"]=False
df.loc[id, "extrMultipleDaysDetails"]=None

In [244]:
id = 2289
df.loc[id, "extrStartDate"] = "2024-09-13"
df.loc[id, "extrEndDate"] = "2024-10-06"

In [245]:
# drop rows with undefined location
flt = df.extrLocationName=="verschiedene Veranstaltungsorte"
df = df[~flt]

In [246]:
cols_to_remove = []
for col in df.columns:
    if col.startswith("OpenAI") or col.startswith("count_") :
        cols_to_remove.append(col)
    
df = df.drop(columns=cols_to_remove, errors="ignore")

In [247]:
flt = (df.isOnMultipleDays==True) 
df_multiple_days = df[flt].copy()

In [248]:
# suppose missng end date in 2 weeks after start 
flt = (df.isOnMultipleDays==True) & (df.extrEndDate.isna() ) 
values = df.loc[flt, "extrStartDate"]
values = (pd.to_datetime(values) + pd.DateOffset(weeks=2)).dt.strftime("%Y-%m-%d")
values
df.loc[flt, "extrEndDate"] = values


In [249]:
# if end date is after 2024-10-31, set it to 2024-10-31
flt = (df.isOnMultipleDays==True) & (pd.to_datetime(df.extrEndDate) > pd.to_datetime("2024-10-31") )
df.loc[flt, "extrEndDate"]="2024-10-31" 

In [250]:
id = 4931
df.loc[id, "extrStartTime"]="20"
df.loc[id, "extrEndTime"]="23"

In [251]:
id = 6155
df.loc[id, "extrEndTime"]="22,5"

In [252]:
id = 3156
df.loc[id, "extrStartTime"]="15"

In [253]:
id= 18582
df.loc[id, "extrStartTime"]="15"
df.loc[id, "extrEndTime"]="16,5"
id=22687
df.loc[id, "extrStartTime"]="15"
df.loc[id, "extrEndTime"]="16,5"


In [254]:
id = 24499
df.loc[id, "extrStartTime"]="13"
df.loc[id, "extrEndTime"]="20,5"

In [255]:
id = 10469
df.loc[id, "extrLocationName"]="Münsterplatz"

In [256]:
id = 18075
df.loc[id, "extrMultipleDaysDetails"]=None

In [ ]:
df.extrStartTime.unique()

In [258]:
flt = df.extrStartTime.str.endswith(":00", na=False)
values = df.loc[flt, "extrStartTime"].str.replace(":00", "")
values
df.loc[flt, "extrStartTime"] = values

In [259]:
flt = df.extrEndTime.str.endswith(":00", na=False)
values = df.loc[flt, "extrEndTime"].str.replace(":00", "")
values
df.loc[flt, "extrEndTime"] = values

In [260]:
flt = df.extrEndTime.str.endswith(":30", na=False)
values = df.loc[flt, "extrEndTime"].str.replace(":30", ",5")
values
df.loc[flt, "extrEndTime"] = values

In [261]:
flt = df.extrEndTime.str.endswith(":45", na=False)
values = df.loc[flt, "extrEndTime"].str.replace(":45", ",5")
values
df.loc[flt, "extrEndTime"] = values

In [262]:
df.extrEndTime = df.extrEndTime.str.replace("19:00 (weekdays), 18:00 (weekends & holidays)", "19")

In [263]:
flt = df.extrStartTime.str.contains("Termin/Uhrzeit", na=False)
values = df.loc[flt, "extrStartTime"].apply(lambda ls: ls.split("\n")[-1]).str.replace(" Uhr", "").str.replace(":00", "").str.replace(":30", ",5")
values
df.loc[flt, "extrStartTime"] = values

In [264]:
df.extrStartTime = df.extrStartTime.str.replace(":00 (for the opening ceremony)", "")

In [ ]:
df.extrStartTime.unique()

In [ ]:
df.extrEndTime.unique()

In [267]:
df.loc[3156, "extrEndTime"]="18"
df.loc[5495, "extrEndTime"]="18"

### estimate the readiness of the dataset

In [ ]:
len(df)

In [ ]:
df.head()

In [270]:
extracted_columns = []
for col in df.columns:
    if col.startswith("extr"):
        extracted_columns.append(col)
    
extracted_columns.append("isOnMultipleDays")

In [ ]:
flt = (df.isOnMultipleDays==False) 
print("total single-day events")
print(len(df[flt]))
for col in extracted_columns:
    flt_2 = df[col].isna()
    print(col+ " has missing rows:")
    print(len(df[flt&flt_2]))

In [ ]:
flt = (df.isOnMultipleDays==True) 
print("total multiple-day events")
print(len(df[flt]))
for col in extracted_columns:
    flt_2 = df[col].isna()
    print(col+ " has missing rows:")
    print(len(df[flt&flt_2]))

In [273]:
#flt = (df.isOnMultipleDays==True) & (df.extrLocationName.isna())  & (df.extrLocationDetails.isna())
#df.loc[flt]

In [274]:
#filename = f"{DATA_FOLDER}df_browse_ai {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl"
#print(filename)
#df.to_pickle(filename)

# transform multiple events into single events

## single day

In [ ]:
df_to_process = df.copy()
df_to_process =  df_to_process.reset_index(names="origin_id")

df_to_process.head()

In [276]:
cols_to_keep = extracted_columns.copy()
cols_to_keep.append("origin_id")
cols_to_keep.append("Event_category")


In [277]:
df_to_process = df_to_process[cols_to_keep]

In [ ]:
df_to_process.head()

In [ ]:
len(df_to_process)

In [280]:
all_cols_except_id = df_to_process.columns.tolist()
all_cols_except_id.remove("origin_id")

In [ ]:
df_to_process.drop_duplicates(inplace=True, subset=all_cols_except_id)
len(df_to_process)

In [ ]:
df_to_process.columns

In [283]:
flt = df_to_process.isOnMultipleDays==False
df_final = df_to_process[flt].copy()
df_to_process = df_to_process[~flt]


In [ ]:
df_to_process.head()

## with existing start and end time

In [ ]:
flt = (~df_to_process.extrStartDate.isna()) & (~df_to_process.extrEndDate.isna()) & (~df_to_process.extrStartTime.isna()) & (~df_to_process.extrEndTime.isna())
df_tmp = df_to_process[flt].copy()
df_to_process = df_to_process[~flt]
df_tmp
# expand multiple day events to single day events
df_tmp["extrStartDate"] = pd.to_datetime(df_tmp["extrStartDate"])
df_tmp["extrEndDate"] = pd.to_datetime(df_tmp["extrEndDate"])
df_tmp


In [ ]:
rows_to_append = []
for i, row in df_tmp.iterrows():
    start_date = row["extrStartDate"]
    end_date = row["extrEndDate"]
    print(f"{start_date=}")
    print(f"{end_date=}")
    date_range = pd.date_range(start_date, end_date)
    
    for date in date_range: # inclusive range
        row_to_append = row.copy()
        row_to_append["extrEndDate"] = None
        row_to_append["extrStartDate"] = date.strftime('%Y-%m-%d')
        rows_to_append.append(row_to_append)
        


In [287]:
df_to_append = pd.DataFrame(rows_to_append)

In [288]:
df_final = pd.concat([df_final, df_to_append])

## with missing time

In [ ]:
with open("../data/keys/OpenAIKey (vkalonova).txt", "r") as file:
        openai_api_key = file.read().strip()

In [296]:
client = OpenAI(api_key=openai_api_key)

In [297]:
system_content = """ Give me a record of date, start and end times of events for each date in which an event happens. Taake weekdays onto account.
Example input:
2024-10-03 till 2024-10-06
MultipleDaysDetails = Thu: 10:00-20:00, Fri: 11:00-19:00, Sat: 10:00-20:00, Sun: 10:00-20:00
Example output:
2024-10-03, 10:00-20:00
2024-10-04, 11:00,-19:00
2024-10-05, 10:00-20:00
2024-10-06, 10:00-20:00
"""

In [ ]:
id = 41
prompt = f""" {df_to_process.loc[id, "extrStartDate"]} till {df_to_process.loc[id, "extrEndDate"]}
MultipleDaysDetails = {df_to_process.loc[id, "extrMultipleDaysDetails"]}
"""
prompt

In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_content},
        {
            "role": "user",
            "content": prompt
        }
    ]
)



In [326]:
openai_respones = []
openai_respones.append({ "id": 41, "response": completion.choices[0].message.content})


In [ ]:
for i, row in df_to_process.iterrows():
    print(i)
    print()
    if i==41:
        continue

    else:
        prompt = f""" {df_to_process.loc[i, "extrStartDate"]} till {df_to_process.loc[i, "extrEndDate"]}
MultipleDaysDetails = {df_to_process.loc[i, "extrMultipleDaysDetails"]}"""
        print(prompt)
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_content},
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )
        res = completion.choices[0].message.content
        print(res)

        openai_respones.append({ "id": i, "response": completion.choices[0].message.content})

        



In [ ]:
df_openai_responses = pd.DataFrame(openai_respones)
df_openai_responses.head()


In [331]:
df_openai_responses["processed_response"] = df_openai_responses.response.apply(lambda x: x.split("\n")).apply(lambda x: [i for i in x if "2024" in i])

In [ ]:
df_openai_responses.head()

### process openai responses

In [ ]:
rows_to_append = []
skipped_rows = 0
openai_explanations = ["Now let's", "To accurately provide", "We need to consider", "To provide a schedule", "Continuing in this ",
                       "Identify", "Check", "To determine", "map these details", "Given the provided input", "If the day is", "To create a schedule",
                       "Note that", "To provide"]
closed_events_escriptions = ["Closed", "No event", "geschlossen", "No event"]
holiday_descriptions = ["Thursday - Tag der Deutschen Einheit", " (Holiday - Labor Day)", 
                        "Thursday, Halloween; not a public holiday", "German Unity Day, public holiday",
                        "Feiertag in Deutschland: Tag der Deutschen Einheit", "Tag der Deutschen Einheit", "(", ")"
                        ]
for _, openai_row in df_openai_responses.iterrows():
    single_dates_responses = openai_row["response"].split("\n")
    # only keep elemets in a list with at least 1 digit

    for res in single_dates_responses:
        res = res.replace("**", "")
        for full_german_weekday in [", Montag, ", ", Dienstag, ", ", Mittwoch, ", ", Donnerstag, ", ", Freitag, ", ", Samstag, ", ", Sonntag, "]:
                res = res.replace(full_german_weekday, ", ")
        for full_german_weekday in [" (Monday): ", " (Tuesday): ", " (Wednesday): ", " (Thursday): ", " (Friday): ", " (Saturday): ", " (Sunday): "]:
                res = res.replace(full_german_weekday, ", ")

        
        """
        for full_weekday in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]:
            res = res.replace(full_weekday, "")
        
        for weekday in ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]:
            res = res.replace(weekday, "")
        res = res.replace(", , ", ", ")
        res = res.replace(" (): ", ", ") """
        for txt in holiday_descriptions:
            res = res.replace(txt, "")
        res = res.replace("  ", " ")

        skip_outer_loop = False
        if not re.search(r"\d", res):
            continue
        for txt in openai_explanations:
            if txt in res:
                skip_outer_loop=True
                break 
        for txt in closed_events_escriptions:
            if txt in res:
                skip_outer_loop=True
                break

        if skip_outer_loop:
            continue
        try:
            date, times1, times2, times3 = res.split(", ")
            start_time1, end_time1 = times1.split("-")
            start_time1 = start_time1.strip().replace(":00", "").replace(":30", ",5")
            end_time1 = end_time1.strip().replace(":00", "").replace(":30", ",5")
            start_time2, end_time2 = times2.split("-")
            start_time2 = start_time2.strip().replace(":00", "").replace(":30", ",5")
            end_time2 = end_time2.strip().replace(":00", "").replace(":30", ",5")
            start_time3 = times3
            end_time3 = None
            id = openai_row["id"]
            assert len(df_to_process[df_to_process.index == id]) == 1
            row_to_append = df_to_process.loc[id].copy()
            row_to_append["extrStartDate"] = date
            row_to_append["extrEndDate"] = None
            row_to_append["extrStartTime"] = start_time1
            row_to_append["extrEndTime"] = end_time1
            rows_to_append.append(row_to_append)
            row_to_append = df_to_process.loc[id].copy()
            row_to_append["extrStartDate"] = date
            row_to_append["extrEndDate"] = None 
            row_to_append["extrStartTime"] = start_time2
            row_to_append["extrEndTime"] = end_time2
            rows_to_append.append(row_to_append)
            row_to_append = df_to_process.loc[id].copy()
            row_to_append["extrStartDate"] = date
            row_to_append["extrEndDate"] = None
            row_to_append["extrStartTime"] = start_time3
            row_to_append["extrEndTime"] = end_time3
            rows_to_append.append(row_to_append)
        except Exception as e:            
            try:
                try:    
                    date, times1, times2 = res.split(", ")
                    start_time1, end_time1 = times1.split("-")
                    start_time1 = start_time1.strip().replace(":00", "").replace(":30", ",5")
                    end_time1 = end_time1.strip().replace(":00", "").replace(":30", ",5")
                    start_time2, end_time2 = times2.split("-")
                    start_time2 = start_time2.strip().replace(":00", "").replace(":30", ",5")
                    end_time2 = end_time2.strip().replace(":00", "").replace(":30", ",5")
                    id = openai_row["id"]
                    assert len(df_to_process[df_to_process.index == id]) == 1
                    row_to_append = df_to_process.loc[id].copy()
                    row_to_append["extrStartDate"] = date
                    row_to_append["extrEndDate"] = None
                    row_to_append["extrStartTime"] = start_time1
                    row_to_append["extrEndTime"] = end_time1
                    rows_to_append.append(row_to_append)
                    row_to_append = df_to_process.loc[id].copy()
                    row_to_append["extrStartDate"] = date
                    row_to_append["extrEndDate"] = None
                    row_to_append["extrStartTime"] = start_time2
                    row_to_append["extrEndTime"] = end_time2
                    rows_to_append.append(row_to_append)

                # write a regexp for dates like this October 1, 2024, 10:00-18:00
                #if re.search(res, r"\d{4}-\d{2}-\d{2}"):

                except Exception as e:
                    date, times = res.split(", ")
                    #print(f"{date=}")
                    #print(f"{times=}")
                    start_time, end_time = times.split("-")
                    start_time = start_time.strip().replace(":00", "").replace(":30", ",5")
                    end_time = end_time.strip().replace(":00", "").replace(":30", ",5")
                    #print(f"{start_time=}")
                    #print(f"{end_time=}")
                    id = openai_row["id"]
                    #print(f"{id=}")
                    assert len(df_to_process[df_to_process.index == id]) == 1
                    row_to_append = df_to_process.loc[id].copy()
                    #print(f"{row_to_append=}")
                    row_to_append["extrStartDate"] = date
                    row_to_append["extrEndDate"] = None
                    row_to_append["extrStartTime"] = start_time
                    row_to_append["extrEndTime"] = end_time
                    rows_to_append.append(row_to_append)
            except Exception as e:
                print(f"could not process {res} because of {e}")
                skipped_rows += 1
                print(f"extrMultipleDaysDetails: {   df_to_process.loc[id, 'extrMultipleDaysDetails']}")
                print()




In [ ]:
len(rows_to_append)

In [ ]:
skipped_rows

In [470]:
df_to_append =  pd.DataFrame(rows_to_append)

In [ ]:
df_to_append.extrEndTime.unique()

In [ ]:
df_to_append.extrStartTime.unique()

In [473]:
flt = df_to_append.extrStartDate.str.startswith("- ")
values = df_to_append.loc[flt, "extrStartDate"].str.replace("- ", "")
df_to_append.loc[flt, "extrStartDate"] = values

In [474]:
for weekday in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]:
    df_to_append.extrStartDate = df_to_append.extrStartDate.str.replace(weekday, "")
    df_to_append.extrEndTime = df_to_append.extrEndTime.str.replace(weekday, "")


In [475]:
df_to_append.extrStartDate =  df_to_append.extrStartDate.str.strip()

In [ ]:
df_to_append.extrStartDate.unique()

In [477]:
flt = df_to_append.extrStartDate.str.contains("2024", na=False)
df_to_append = df_to_append[flt]


In [ ]:
len(df_to_append)

In [479]:
df_to_append.extrStartTime = df_to_append.extrStartTime.str.replace(":30", ",5")

In [480]:
flt = df_to_append.extrStartTime.str.contains("Feiertag")
df_to_append = df_to_append[~flt]

In [ ]:
df_to_append.extrStartTime.value_counts()

In [ ]:
df_to_append.extrEndTime.value_counts()

In [484]:
flt = df_to_append.extrEndTime.str.contains("Closing", na=False)
df_to_append = df_to_append[~flt]

In [ ]:
len(df_to_append)

In [486]:
df_final = pd.concat([df_final, df_to_append])

## save

In [ ]:
len(df_final)

In [489]:


df_final = df_final.drop_duplicates()

In [ ]:
len(df_final)

In [493]:
df_final = df_final.dropna(how="all", axis=1)

In [ ]:
len(df_final)

In [ ]:
df_final.tail()

In [498]:
df_final["Source"]="browse_ai"

In [ ]:
# df.to_excel(f"{DATA_FOLDER}df_browse_ai {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.xlsx")

In [499]:
df_final.to_csv(f"{DATA_FOLDER}df_browse_ai final {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv")

In [500]:
df_final.to_pickle(f"{DATA_FOLDER}df_browse_ai final {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl")